# Transcript A for Lecture, February 16, 2023

In [1]:
import sys

########################################
# Change the string in the line below! #
########################################
sys.path.append("/Users/gilbert/Documents/CS111-2023-winter/Python") 

import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import linalg as spla
import scipy.sparse
import scipy.sparse.linalg
from scipy import integrate
import networkx as nx
import cs111

##########################################################
# If this import for matplotlib doesn't work, try saying #
#   conda install -c conda-forge ipympl                  #
# at a shell prompt on your computer                     #
##########################################################
import matplotlib
# %matplotlib inline
%matplotlib tk

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d

np.set_printoptions(precision = 4)

# Image compression using SVD and low-rank approximation

The low-rank approximation theory above is useful in many contexts for summarizing or compressing data. For example, "latent semantic indexing" is a summarization and classification technique for word in documents; you can read more about it at https://en.wikipedia.org/wiki/Latent_semantic_analysis.

Here we will give an example of using SVD to compress the data in an image. We start with a famous photograph of Yosemite by Ansel Adams.

In [2]:
# Read the image from a .jpg file and get just the black intensity of each pixel
adams = plt.imread('02-16-halfdome.jpg')
M = np.float64(adams[:,:,0])
nrows, ncols = M.shape
print('size of matrix M:', M.shape)

size of matrix M: (1200, 956)


In [3]:
# plot the original image (matrix)
plt.figure(figsize=(10,10))
plt.gray()
plt.imshow(M)
plt.title('original image, %d by %d pixels' % (nrows, ncols))
print()

The matrix $M$ has about a million elements, all independent of each other. Here are a few of them.

In [4]:
nrows, ncols = M.shape
print('Storage needed for all of M:', nrows*ncols)
print()
print('First 5 rows and cols of M:\n', M[:5,:5])

Storage needed for all of M: 1147200

First 5 rows and cols of M:
 [[93. 91. 84. 82. 87.]
 [85. 85. 84. 83. 82.]
 [80. 80. 82. 83. 81.]
 [87. 86. 83. 82. 84.]
 [77. 82. 78. 78. 87.]]


Let's compute the SVD of $M$. We plot the singular values on a log scale.

In [5]:
U, sigma, Vt = spla.svd(M)

# %matplotlib inline
fig0 = plt.figure(figsize=(10,8))
plt.plot(sigma,'.')
plt.yscale('log')
plt.title('Singular values on log scale')
plt.ylabel('singular value')
plt.xlabel('index')
plt.grid()
print()

We notice that the sizes of the singular values drop by two orders of magnitude in the first 150 or so. That suggests that the rank-150 approximation to $M$ is accurate to within about $1\%$. The advantage of the approximation is that we don't need to store all of $A$; we can reconstruct the image if we just store the first 150 vectors $u_i$ and $v_i$ (we can either store the first 150 $\sigma_i$ too, or we can multiply them into the $u_i$'s or $v_i$'s).

Let's try it. We'll also verify that the 2-norms are what Theorem 7 says they should be.

In [6]:
k = 150 # try this first with 150, then with 50, then 10, then 2, then 1

print(f"Rank {k} approximation:")
print()

nrows, ncols = M.shape
Mk = np.zeros(M.shape)
for i in range(k):
    Mk += sigma[i] * np.outer(U[:,i], Vt[i,:])
    
print('2-norm(M) =', npla.norm(M,2))
print(' sigma[0] =', sigma[0])
print()
print('2-norm(M%d-M) =' % k, npla.norm(Mk-M, 2))
print('    sigma[%d] =' % k, sigma[k])
print()
print('Relative error =', sigma[k]/sigma[0])
print()

Mstorage = nrows*ncols
Mkstorage = k * (nrows+ncols)

print('Storage needed for all of M:', Mstorage)
print('Storage needed for M%d:     ' % k, Mkstorage)
print()
print('Compression factor:', Mstorage/Mkstorage)


Rank 150 approximation:

2-norm(M) = 83403.20249998092
 sigma[0] = 83403.20249998092

2-norm(M150-M) = 943.3802639594555
    sigma[150] = 943.3802639594556

Relative error = 0.011311079619031073

Storage needed for all of M: 1147200
Storage needed for M150:      323400

Compression factor: 3.547309833024119


Finally let's look at the compressed image, which is $M_k$. You see that we do give up some image quality for the compression. 

Try changing the value of $k$ in the cell above, and running this experiment with a few different values to compare. How much can you compress without being able to see the difference between $M$ and $M_k$? At the other end, if you just take $k=1$ or $4$ or $10$ can you see any features of the original image in the compressed version?

In [7]:
# plot the compressed image
plt.figure(figsize=(10,10))
plt.gray()
plt.imshow(Mk)
plt.title('compressed image, rank %d' % k)
print()

In [8]:
k = 50 # try this first with 150, then with 50, then 10, then 2, then 1

print(f"Rank {k} approximation:")
print()

nrows, ncols = M.shape
Mk = np.zeros(M.shape)
for i in range(k):
    Mk += sigma[i] * np.outer(U[:,i], Vt[i,:])
    
print('2-norm(M) =', npla.norm(M,2))
print(' sigma[0] =', sigma[0])
print()
print('2-norm(M%d-M) =' % k, npla.norm(Mk-M, 2))
print('    sigma[%d] =' % k, sigma[k])
print()
print('Relative error =', sigma[k]/sigma[0])
print()

Mstorage = nrows*ncols
Mkstorage = k * (nrows+ncols)

print('Storage needed for all of M:', Mstorage)
print('Storage needed for M%d:     ' % k, Mkstorage)
print()
print('Compression factor:', Mstorage/Mkstorage)

Rank 50 approximation:

2-norm(M) = 83403.20249998092
 sigma[0] = 83403.20249998092

2-norm(M50-M) = 2055.4835155806973
    sigma[50] = 2055.4835155806995

Relative error = 0.024645138963113194

Storage needed for all of M: 1147200
Storage needed for M50:      107800

Compression factor: 10.641929499072356


In [9]:
# plot the compressed image
plt.figure(figsize=(10,10))
plt.gray()
plt.imshow(Mk)
plt.title('compressed image, rank %d' % k)
print()

In [12]:
k = 6 # try this first with 150, then with 50, then 10, then 2, then 1

print(f"Rank {k} approximation:")
print()

nrows, ncols = M.shape
Mk = np.zeros(M.shape)
for i in range(k):
    Mk += sigma[i] * np.outer(U[:,i], Vt[i,:])
    
print('2-norm(M) =', npla.norm(M,2))
print(' sigma[0] =', sigma[0])
print()
print('2-norm(M%d-M) =' % k, npla.norm(Mk-M, 2))
print('    sigma[%d] =' % k, sigma[k])
print()
print('Relative error =', sigma[k]/sigma[0])
print()

Mstorage = nrows*ncols
Mkstorage = k * (nrows+ncols)

print('Storage needed for all of M:', Mstorage)
print('Storage needed for M%d:     ' % k, Mkstorage)
print()
print('Compression factor:', Mstorage/Mkstorage)

Rank 6 approximation:

2-norm(M) = 83403.20249998092
 sigma[0] = 83403.20249998092

2-norm(M6-M) = 7574.449642160659
    sigma[6] = 7574.449642160661

Relative error = 0.09081725179752412

Storage needed for all of M: 1147200
Storage needed for M6:      12936

Compression factor: 88.68274582560296


In [13]:
# plot the compressed image
plt.figure(figsize=(10,10))
plt.gray()
plt.imshow(Mk)
plt.title('compressed image, rank %d' % k)
print()

In [ ]:
# plot the original image (matrix)
plt.figure(figsize=(10,10))
plt.gray()
plt.imshow(M)
plt.title('original image, %d by %d pixels' % (nrows, ncols))
print()

In [14]:
plt.close('all')